
# Callin Switzer
Read in all csv files from individual trials for heavy/light flower experiment, and 
combine into a single long dataset, combined with metadata 


In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
import pandas as pd
import time
import re
from datetime import datetime
import sys
import csv
import seaborn as sns
from collections import Counter

print(sys.version, "\n")

now = datetime.now()
print("last run on " + str(now))

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:04:09) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)] 

last run on 2017-12-07 17:20:05.377936


In [4]:
# define directories
baseDir = os.getcwd()
dataDir = '/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/SonicationBehavior_HeavyLight/'
metadataDir1 = '/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/BeeMetaData/'

In [5]:
baseDir

'/Users/cswitzer/Documents/GitRepos/BeeSonicationLearning'

### Read in bee metadata

In [8]:
metadataFile = metadataDir1 + "HeavyLight_BeeMetaData_cleaned.csv"

# read in data
mdta = pd.read_csv(metadataFile, header = 0, index_col = None)

In [9]:
mdta.head()

,BeeColorNum,IT,Hive,FlowerType1,FlowerType2,trainingFile_,dateFrozenOrMarked
0,gold,4.13,3,weighted,sham,gold_26Sept_Weghted_Unweighted/2016_09_26__17_...,26-Sep-16
1,1,4.07,4,weighted,sham,Bee1_27Sept_Hive4_W_S/2016_09_27__10_52_05_amp...,27-Sep-16
2,2,4.42,4,sham,weighted,Bee2_27Sept_Hive4_S_W/2016_09_27__11_54_44_amp...,27-Sep-16
3,3,4.27,3,sham,weighted,Bee3_28Sept_hive3_S_W/2016_09_28__08_33_35_amp...,28-Sep-16
4,4,4.27,3,weighted,sham,Bee4_28Sept2016_Hive3_W_S/2016_09_28__08_45_02...,28-Sep-16


In [35]:

NoFlightFolder = "/Users/cswitzer/Dropbox/SonicationBehavior/SonBehData/SonicationBehavior_HeavyLight_NoFlights/"

folders = mdta.trainingFile_

# define subfolders
fldrs2 = [folders[ii].split("/")[0] for ii in range(len(folders))]

for ii in range(len(folders)):
    tmpDF = pd.read_table(os.path.join(dataDir, folders[ii]), header = None, sep = ',')

    # add folder name to data frame
    tmpDF[7] = folders[ii]
    fcontents = [f for f in os.listdir(os.path.join(dataDir,fldrs2[ii])) if not f.startswith('.')]
    folderInFolder = [x for x in fcontents if not 'ampFreq.txt' in x]

    # list files in inner folder
    accRecFiles = [x for x in os.listdir(os.path.join(dataDir, fldrs2[ii], folderInFolder[0])) if not x.startswith('.')]
    accRecFiles.sort()
    tmpDF[8] = accRecFiles

    # remove wingbeats
    tmpDF2 = tmpDF.loc[(tmpDF[0] > 220) & (tmpDF[0] < 450)]

    # renumber index
    tmpDF2.index = (np.arange(1, tmpDF2.shape[0] + 1))

    # write to .csv
    tmpDF2.to_csv(NoFlightFolder + fldrs2[ii] + '.csv', 
                 header = False, index = True)
    print(ii)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [37]:
## combine csv's all into a single file
csvDir = NoFlightFolder

csvFiles = [f for f in os.listdir(csvDir) if f.endswith('.csv')]

np_array_list = []
for file_ in csvFiles:
    df = pd.read_csv(os.path.join(csvDir, file_),index_col=None, header=None)
    np_array_list.append(df.as_matrix())

comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)

big_frame.columns = ['index', 'freq', 'amp', 'datetime', 'rewNum', 'rewTF', 'lowRewAmp', 'highrewAmp', 'BeeNumCol', 'accFile']

print(big_frame.shape)
big_frame.head()

(2360, 10)


,index,freq,amp,datetime,rewNum,rewTF,lowRewAmp,highrewAmp,BeeNumCol,accFile
0,1,250,0.17945,2016_09_28__09_59_13_112,1,T,0,5,Bee7_28Sept2016_Hive3_S_W/2016_09_28__09_58_58...,2016_09_28__09_59_13_112_220_450_test.txt
1,2,300,0.56657,2016_09_28__09_59_14_520,2,T,0,5,Bee7_28Sept2016_Hive3_S_W/2016_09_28__09_58_58...,2016_09_28__09_59_14_520_220_450_test.txt
2,3,400,0.68371,2016_09_28__09_59_15_030,3,T,0,5,Bee7_28Sept2016_Hive3_S_W/2016_09_28__09_58_58...,2016_09_28__09_59_15_030_220_450_test.txt
3,4,410,0.80118,2016_09_28__09_59_15_707,4,T,0,5,Bee7_28Sept2016_Hive3_S_W/2016_09_28__09_58_58...,2016_09_28__09_59_15_707_220_450_test.txt
4,5,370,0.52666,2016_09_28__09_59_16_277,5,T,0,5,Bee7_28Sept2016_Hive3_S_W/2016_09_28__09_58_58...,2016_09_28__09_59_16_277_220_450_test.txt


In [ ]:
# here

In [ ]:
# plot histogram of frequencies
vls = [int(big_frame["freq"][ii]) for ii in range(len(big_frame))]
plt.hist(vls, bins = 50)
plt.show()

In [ ]:
# check to see if datetime and accFile agree
eqs = [big_frame['datetime'][ii][1:25] == big_frame['accFile'][ii][0:24] for ii in range(len(big_frame['datetime']))]

In [ ]:
np.sum(np.invert(eqs)) # should be zero if all of them match

In [ ]:
# find the one that doesn't match
big_frame.loc[np.invert(eqs)] # now they all match

In [ ]:
str2 = [big_frame['BeeNumCol'][ii].split("Bee")[1] for ii in range(len(big_frame['BeeNumCol']))]

In [ ]:
beeColNum = [str2[ii].split(r'_')[0] for ii in range(len(str2))]

In [ ]:
hiveNum = [str2[ii].split(r'_')[2] for ii in range(len(str2))]

In [ ]:
hive = [int(re.findall(r'\d+', ii)[0]) for ii in hiveNum]

In [ ]:
# function to get only characters
def extractChar(string):
    return(" ".join(re.findall("[a-zA-Z]+", string)))

In [ ]:
# get just bee color
beeCol = [extractChar(strg) for strg in beeColNum]

In [ ]:
# get datetime
dateTime = [datetime.strptime(dt1, ' %Y_%m_%d__%H_%M_%S_%f') for dt1 in big_frame['datetime']]

In [ ]:
# string format time
dateTime_format = [datetime.strftime(datetime_object, "%Y-%m-%d %H:%M:%S.%f") for datetime_object in dateTime]

In [ ]:
# get reward frequencies
s1 = big_frame['accFile'][0]

In [ ]:
rewFrqs = [s1.split("_")[8:10] for s1 in big_frame['accFile']]

In [ ]:
# print system info
import IPython
print(IPython.sys_info())

In [ ]:
# show installed packages and versions
!pip freeze 

In [ ]:
# convert to html, so ppl don't have to run python to see code
os.chdir(baseDir)
!jupyter nbconvert --to html 004_CombineTrialsHeavyLightData